# Проект для «Викишоп» c BERT

# Введение

### Описание проекта 
**В данном проекте будут рассмотрены ДВА варианта подготовки признаков для обучения моделей.**  

Проект нацелен на воплощение алгоритма определения токсичных комментариев для их отправки на модерацию.
### Цель проекта
Создать модель классификации комментариев на позитивные и негативные.
### Описание данных
**Нам предоставлен набор данных с разметкой о токсичности правок от комании «Викишоп».**   


**Признаки:**
- text - текст комментария

**Целевой признак:**
- toxic — категория содержимого в комментарии

**Заказчику важны:**

- значение метрики качества F1 не меньше 0.75

### План работы
1. [Загрузка данных](#section_1)  
2. [Обучение и анализ моделей](#section_2)
3. [Проверка итоговой модели](#section_3)
4. [Общий вывод](#section_4)

<a id='section_1'></a>
## Загрузка данных

In [1]:
# !pip install transformers

Загрузим необходимые библиотеки.

In [2]:
# работа с данными
import pandas as pd
import numpy as np
import os
import torch
import transformers

# модели машинного обучения
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from transformers import AutoTokenizer, AutoModel

# вспомогательные средства 
from tqdm import notebook
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from nltk.corpus import wordnet

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Egor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Egor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Egor\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Egor\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Egor\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

Произведём загрузку датасета.

In [3]:
path_1 = r'C:\Projects\wikishop\toxic_comments.csv'
path_2 = '/datasets/toxic_comments.csv'

if os.path.exists(path_1):
    df = pd.read_csv(path_1, index_col=[0]).reset_index(drop=True)
elif os.path.exists(path_2):
    df = pd.read_csv(path_2, index_col=[0]).reset_index(drop=True)
else:
    print('Something is wrong')

In [4]:
df = df.head(30000)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    30000 non-null  object
 1   toxic   30000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 468.9+ KB


In [6]:
df.head(10)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


In [7]:
df['toxic'].mean()

0.1042

В данных присутствует дисбаланс классов.

## Подготовка данных c BERT

Выполним загрузку модулей для работы с BERT.

In [8]:
from transformers import AutoTokenizer, AutoModel

model_name = 'unitary/toxic-bert'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# инициализируем токенизатор
default_tokenizer = AutoTokenizer.from_pretrained(model_name)

# инициализируем модель
default_model = AutoModel.from_pretrained(model_name)

default_model = default_model.to(device)

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
default_model.device

device(type='cuda', index=0)

In [10]:
df_tokenized = default_tokenizer(list(df['text']), return_tensors='pt', truncation=True, padding=True, max_length=128)

In [11]:
df_tokenized = df_tokenized.to(device)

In [12]:
torch.cuda.empty_cache()

In [13]:
batch_size = 200
embeddings = []
torch.cuda.empty_cache()
for i in notebook.tqdm(range(df_tokenized['input_ids'].shape[0] // batch_size)):
        batch = torch.cuda.LongTensor(df_tokenized['input_ids'][batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.cuda.LongTensor(df_tokenized['attention_mask'][batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = default_model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].cpu())

  0%|          | 0/150 [00:00<?, ?it/s]

Произведём объединение эмбеддингов.

In [14]:
features = torch.concatenate(embeddings)

Разделим данные на тренировочную и тестовую выборки.

In [36]:
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    df['toxic'][:features.shape[0]], 
    test_size=0.2, 
    random_state=12345
)

In [37]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

(torch.Size([24000, 768]), (24000,), torch.Size([6000, 768]), (6000,))

## Подготовка данных TF-IDF

In [17]:
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize(text):
    word_list = nltk.word_tokenize(text)
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in word_list])
    return lemmatized_output

def clear_text(text):
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    return " ".join(text.split())

In [9]:
import sys

print(sys.getdefaultencoding())
sys.getsizeof('asd'), sys.getsizeof(u'asd'), sys.getsizeof(str.encode('asd', 'ascii'))

utf-8


(52, 52, 36)

In [19]:
corpus = df['text'].head(30000)

In [20]:
corpus_lemm = [lemmatize(clear_text(corpus[i])) for i in notebook.tqdm(range(len(corpus)))]

  0%|          | 0/30000 [00:00<?, ?it/s]

In [21]:
stopwords = set(nltk_stopwords.words('english'))

In [27]:
X_train_tf, X_test_tf, y_train_tf, y_test_tf = train_test_split(corpus_lemm, df['toxic'], test_size=0.2, random_state=12345)

print(f"Размер тренировочной выборки: {len(X_train)}")
print(f"Размер тестовой выборки: {len(X_test)}")

Размер тренировочной выборки: 24000
Размер тестовой выборки: 6000


In [28]:
vectorizer = TfidfVectorizer(stop_words=stopwords)

X_train_vector = vectorizer.fit_transform(X_train_tf)
X_test_vector = vectorizer.transform(X_test_tf)

### Обучение логистической регрессии на преобразованных данных методом TF-IDF

In [29]:
model = LogisticRegression()

param_grid = {
    'solver' : ['liblinear'], 
    'penalty' : ['l2'], 
    'C' : [10, 1.0, 0.1]
}

lrgs = GridSearchCV(
    estimator=model, 
    param_grid=param_grid,  
    cv=3, 
    scoring='f1',
    #n_jobs=-1,
    verbose=0
)

lrgs.fit(X_train_vector, y_train_tf)

print(f"F1 cross-val score: {lrgs.best_score_:.3f}")
print(f"Best params: {lrgs.best_params_}")

F1 cross-val score: 0.688
Best params: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}


### Проверка модели на тестовой выборке

In [30]:
model = RandomForestClassifier(random_state=12345)

model.fit(X_train_vector, y_train_tf)
y_pred = model.predict(X_test_vector)
f1_score(y_test_tf, y_pred)

0.6996966632962589

Для экономии времени данная модель обучалась только на части всего датасета, если производить обучение на полных данных, то модель линейной регрессии по качеству превысит заданный уровень F1, но всё ещё будет значительно отставать от моделей, обученных на данных, преобразованных с помощью BERT, результаты приведены ниже.

<a id='section_2'></a>
## Обучение и анализ моделей

### LogisticRegression

In [39]:
model = LogisticRegression()

param_grid = {
    'solver' : ['liblinear'], 
    'penalty' : ['l2'], 
    'C' : [10, 1.0, 0.1]
}

lrgs = GridSearchCV(
    estimator=model, 
    param_grid=param_grid,  
    cv=3, 
    scoring='f1',
    n_jobs=-1,
    verbose=0
)

lrgs.fit(X_train, y_train)

print(f"F1 cross-val score: {lrgs.best_score_:.3f}")
print(f"Best params: {lrgs.best_params_}")

F1 cross-val score: 0.931
Best params: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}


### RandomForest

In [40]:
model = RandomForestClassifier(random_state=12345)

params = {
    'max_depth': [1, 5, 10, 15],
    'max_features': ['auto', 'sqrt', 'log2'],
    'n_estimators': [10, 50, 100],
}

rf_hrs = HalvingRandomSearchCV(
    model,
    params,
    cv=3,
    scoring='f1',
    error_score='raise',
    n_jobs=-1,
    verbose=0,
    random_state=12345
)

rf_hrs.fit(X_train, y_train)
print(f"F1 cross-val score: {rf_hrs.best_score_:.3f}")
print(f"Best params: {rf_hrs.best_params_}")

C:\Anaconda\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 36 is smaller than n_iter=2000. Running 36 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


F1 cross-val score: 0.921
Best params: {'n_estimators': 100, 'max_features': 'auto', 'max_depth': 1}


### LightGBM

In [45]:
model = LGBMClassifier()

params = {
    'n_estimators': [50, 100],
    'max_depth': [3, 10, 15, 25],
    'num_leaves': [7, 15, 35],
    'learning_rate': [0.1, 0.2, 0.5, 0.7]
}

lgbm_hrs = HalvingRandomSearchCV(
    model, 
    params, 
    cv=3, 
    scoring='f1',
    n_jobs=-1,
    verbose=0,
    random_state=12345
)

lgbm_hrs.fit(X_train, y_train)

print(f"F1 cross-val score: {lgbm_hrs.best_score_:.3f}")
print(f"Best params: {lgbm_hrs.best_params_}")

C:\Anaconda\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 96 is smaller than n_iter=2000. Running 96 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


F1 cross-val score: 0.879
Best params: {'num_leaves': 15, 'n_estimators': 50, 'max_depth': 3, 'learning_rate': 0.1}


In [46]:
results = pd.DataFrame(
    data=[lrgs.best_score_, rf_hrs.best_score_, lgbm_hrs.best_score_], 
    index=['LinearRegression', 'RandomForest', 'LightGBM'], 
    columns=['F1_score']
)

results.style.format('{:.3f}')

,F1_score
LinearRegression,0.931
RandomForest,0.921
LightGBM,0.879


Как видно из таблицы, лучший результат на кросс-валидации показала модель с использованием алгоритма RandomForest. Также стоит заметить, что все модели прошли заданный порог качества.

<a id='section_3'></a>
## Проверка итоговой модели

Проверим качество модели случайного леса на тестовой выборке.

In [43]:
y_pred = rf_hrs.predict(X_test)

print(f"F1 score: {f1_score(y_test, y_pred):.3f}")
print(f"accuracy score: {accuracy_score(y_test, y_pred):.3f}")

F1 score: 0.937
accuracy score: 0.987


Проверим модель на адекватность с помощью простейшей константной модели.

In [44]:
dummy = DummyClassifier(strategy='constant', constant=1)
dummy.fit(X_train, y_train)

y_pred = dummy.predict(X_test)

print(f"Dummy F1 score: {f1_score(y_test, y_pred):.3f}")
print(f"Dummy accuracy score: {accuracy_score(y_test, y_pred):.3f}")

Dummy F1 score: 0.184
Dummy accuracy score: 0.101


Модель прошла проверку на адекватность. Сравнивать модель с константной моделью по F1 бессмысленно, так как в её предсказаниях отсутствуют верноположительные предсказания.

<a id='section_4'></a>
## Общий вывод

В ходе исследования были обработаны и проанализированы тексты комментариев сервиса «Викишоп». В качестве основного инструмента выступила предобученная модель BERT, которая преобразовла тексты в эмбеддинги с использованием вычислений на графическом процессоре. Созданы модели классификации комментариев. Все построенные модели показали удовлетворяющий условию задачи результат.
После проделанной работы можно выделить алгоритм RandomForest, который продемонстрировал лучший результат в списке моделей машинного обучения. Но так же стоит отметить, что все модели показали качество по метрике F1 значительно превышающее 0,75. 

**После проделанного исследования можно сделать следующий вывод:**

1. Для классификации комментариев целесообразно использовать модель RandomForest.